In [1]:
download_directory = 'https://mhealth.md2k.org/images/datasets/mORALdataset09072020.zip'
destination_directory = './data/'
import requests, zipfile, io
r = requests.get(download_directory)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(destination_directory)

In [3]:
zipfile_directory = './data/mORAL_dataset_for_python_upload_09072020/data.zip'
destination_directory = './data/mORAL_dataset_for_python_upload_09072020/raw_data/'
with zipfile.ZipFile(zipfile_directory) as zf:
        zf.extractall(destination_directory)

In [73]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from joblib import Parallel,delayed
import pickle
sensor_data_directory = './data/mORAL_dataset_for_python_upload_09072020/raw_data/data/sensor_data/'
all_participants_directory = [os.path.join(sensor_data_directory,a) for a in os.listdir(sensor_data_directory) if len(a)<=4]

In [68]:
def groupby_window_size(df):
    if df.shape[0]<100:
        return pd.DataFrame([],columns=['user','timestamp','starttime','endtime','data'])
    df = df.sort_values('timestamp').reset_index(drop=True)
    df['magnitude'] = df.apply(lambda a:np.linalg.norm([a['x'],a['y'],a['z']]),axis=1)
    if df['magnitude'].std()<=not_wearing_standard_deviation:
        return pd.DataFrame([],columns=['user','timestamp','starttime','endtime','data'])
    user = df['user'].values[0]
    timestamp = df['timestamp'].values[0]
    starttime = df['time'].min()
    endtime = df['time'].max()
    data = df[['time','x','y','z']].values
    return pd.DataFrame([[user,timestamp,starttime,endtime,data]],columns=['user','timestamp','starttime','endtime','data'])

wrist = 'right'
window_size = 20
not_wearing_standard_deviation = 0.001174897554939529
all_data_participant = []
for participant_directory in all_participants_directory:
    for day in os.listdir(participant_directory):
        day_directory = os.path.join(participant_directory,day)
        try:
            right_aclx = pd.read_csv(os.path.join(day_directory,'{}-wrist-accelx.csv'.format(wrist)),names=['time','x'])
            right_acly = pd.read_csv(os.path.join(day_directory,'{}-wrist-accely.csv'.format(wrist)),names=['time','y'])
            right_aclz = pd.read_csv(os.path.join(day_directory,'{}-wrist-accelz.csv'.format(wrist)),names=['time','z'])
        except FileNotFoundError:
            print(day_directory)
            continue
        data = right_aclz.merge(right_acly,on='time', how='inner').merge(right_aclx,on='time', how='inner')
        user = participant_directory.split('/')[-1]
        data['user'] = [user]*data.shape[0]
        data['timestamp'] = data['time'].apply(lambda a:datetime.utcfromtimestamp(a/1000))
        grouped_data = pd.concat(Parallel(n_jobs=-1,verbose=0)(delayed(groupby_window_size)(df) for i,df in data.groupby(pd.Grouper(key='timestamp',freq='{}S'.format(window_size)),as_index=False)))
        all_data_participant.append(grouped_data)

./data/mORAL_dataset_for_python_upload_09072020/raw_data/data/sensor_data/9a6b/d2017-04-28
./data/mORAL_dataset_for_python_upload_09072020/raw_data/data/sensor_data/93a2/d2018-06-22
./data/mORAL_dataset_for_python_upload_09072020/raw_data/data/sensor_data/896d/d2018-05-08
./data/mORAL_dataset_for_python_upload_09072020/raw_data/data/sensor_data/896d/d2018-05-05
./data/mORAL_dataset_for_python_upload_09072020/raw_data/data/sensor_data/896d/d2018-05-03
./data/mORAL_dataset_for_python_upload_09072020/raw_data/data/sensor_data/896d/d2018-05-06
./data/mORAL_dataset_for_python_upload_09072020/raw_data/data/sensor_data/896d/d2018-05-04
./data/mORAL_dataset_for_python_upload_09072020/raw_data/data/sensor_data/896d/d2018-05-02
./data/mORAL_dataset_for_python_upload_09072020/raw_data/data/sensor_data/896d/d2018-05-09
./data/mORAL_dataset_for_python_upload_09072020/raw_data/data/sensor_data/89f4/d2018-12-28
./data/mORAL_dataset_for_python_upload_09072020/raw_data/data/sensor_data/89f4/d2018-12-09

In [70]:
all_data = pd.concat(all_data_participant)

In [74]:
pickle.dump(all_data,open('./data/mORAL_dataset_for_python_upload_09072020/processed_data/all_data_{}_wrist.p'.format(wrist),'wb'))